In [1]:
import os

OPENAI_API_KEY = open(os.getcwd() + '/keys/gpt4.key', 'r').readline().rstrip()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
from typing import List
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.chat_models import ChatOpenAI

In [3]:
chat = ChatOpenAI(model_name="gpt-4", temperature=0.)

In [4]:
class Plan(BaseModel):
  tasks: List[str] = Field(description="list of all tasks that the robot has to carry out")

parser = PydanticOutputParser(pydantic_object=Plan)

In [5]:
template="""
You are a helpful assistant in charge of controlling a robot manipulator.
Your task is that of creating a full plan of what the robot has to do in order to stack 4 cubes on top of each other.
This is the scene description:
  - The robot starts at the home location with the gripper open.
  - cube_1 should not be move so you have to move cube_2, cube_3 and cube_4 on top of cube_1.
  - Always stack a new cube on top of the last cube you have put on top of the stack.

You can control the robot in the following way:
  1. move the gripper of the robot
  2. open the gripper
  3. close the gripper

{format_instructions}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
#human_template="{text}"
#human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])#, human_message_prompt])

In [6]:
output = chat(chat_prompt.format_prompt(format_instructions=parser.get_format_instructions()).to_messages())

In [8]:
print(output.content)

{
  "tasks": [
    "move gripper to cube_2",
    "close gripper",
    "move gripper to cube_1",
    "open gripper",
    "move gripper to cube_3",
    "close gripper",
    "move gripper to cube_2",
    "open gripper",
    "move gripper to cube_4",
    "close gripper",
    "move gripper to cube_3",
    "open gripper"
  ]
}


In [7]:
parser.parse(output.content)

Plan(tasks=['move gripper to cube_2', 'close gripper', 'move gripper to cube_1', 'open gripper', 'move gripper to cube_3', 'close gripper', 'move gripper to cube_2', 'open gripper', 'move gripper to cube_4', 'close gripper', 'move gripper to cube_3', 'open gripper'])